# 1. Import Modules

In [ ]:
import os
from dotenv import load_dotenv
import requests as r
import json
import pandas as pd
import pymysql as sql
from sqlalchemy import create_engine

In [ ]:
# reading the contents of the .env file stored in the notebook's folder
load_dotenv()

# 2. The First GET Request

### 2.1 API Keys, Query Parameters, and the Endpoint URL

In [ ]:
api_key = os.getenv('api_key')
channel_id = os.getenv('channel_id')

In [ ]:
endpoint_search = 'https://www.googleapis.com/youtube/v3/search'

In [ ]:
# parameters for the first request
page_token = ''
part_search = 'snippet,id'
order = 'date'
maxResults = '50'
q = 'messi'
type = 'video'
videoDefinition = 'high'

params_search = {
    'key' : api_key, 
    'channelid' : channel_id, 
    'part' : part_search, 
    'order' : order, 
    'maxResults' : maxResults, 
    'page_token' : page_token,
    'q' : q,
    'type' : type,
    'videoDefinition' : videoDefinition
}

### 2.2 The GET Request

In [ ]:
response_search = r.get(endpoint_search, params = params_search)

if response_search.status_code == 200:
    print('Successfully connected!', 'Status Code:', response_search.status_code)
else:
    print('Something went wrong!', 'Status Code: ', response_search.status_code)

### 2.3 Convert the Output to JSON format and Then Flatten It

In [ ]:
response_search = response_search.json()
data_search = pd.json_normalize(response_search, record_path = ['items'], meta = ['kind', 'etag', 'nextPageToken', 'regionCode'], meta_prefix = 'meta')
data_search.head(1)

### 2.4 Delete / Rename the Columns

In [ ]:
data_search = data_search[['id.kind', 'id.videoId', 'snippet.publishedAt', 'snippet.channelId', 'snippet.title', 'snippet.description', 'snippet.channelTitle', 'metanextPageToken', 'metaregionCode']]

In [ ]:
data_search = data_search.rename(
    columns = {
        'id.kind':'kind', 
        'id.videoId':'video_id', 
        'snippet.publishedAt':'published_at', 
        'snippet.channelId':'channel_id',
        'snippet.title':'title',
        'snippet.description':'description',
        'snippet.channelTitle':'channel_title',
        'metanextPageToken':'next_page_token',
        'metaregionCode':'region_code'
    }
)

# 3. The Second GET Request

### 3.1 API Keys, Query Parameters, and the Endpoint URL

In [ ]:
endpoint_videos = 'https://www.googleapis.com/youtube/v3/videos'

In [ ]:
# parameters for the second request

part_videos = 'statistics'
video_id = data_search['video_id']

params_videos = {
    'key' : api_key, 
    'id' : video_id, 
    'part' : part_videos,
    'maxResults' : maxResults,
    'page_token' : page_token
}

### 3.2 The GET Request

In [ ]:
response_videos = r.get(endpoint_videos, params = params_videos)

if response_videos.status_code == 200:
    print('Successfully connected!', 'Status Code:', response_videos.status_code)
else:
    print('Something went wrong!', 'Status Code: ', response_videos.status_code)

### 3.3 Convert the Output to JSON format and Then Flatten It

In [ ]:
response_videos = response_videos.json()
data_videos = pd.json_normalize(response_videos, record_path = ['items'])
data_videos.head(1)

### 3.4 Delete / Rename the Columns

In [ ]:
data_videos = data_videos[['id', 'statistics.viewCount', 'statistics.likeCount', 'statistics.favoriteCount', 'statistics.commentCount']]

In [ ]:
data_videos = data_videos.rename(
    columns = {
        'id':'video_id',
        'statistics.viewCount':'view_count', 
        'statistics.likeCount':'like_count', 
        'statistics.favoriteCount':'favorite_count', 
        'statistics.commentCount':'comment_count'
    }
)

# 4. Joining the Datasets

In [ ]:
# the video_id column is being used to join the 2 datasets
data_final = pd.merge(data_search, data_videos, how = 'inner', on = 'video_id')
data_final.head(1)

# 5. Deleting Records With Missing Values & Fixing Data Types

In [ ]:
# checking the n/a values and data types
data_final.info()

In [ ]:
# deleting records with n/a values
data_final = data_final.dropna(subset = ['like_count'])
data_final = data_final.dropna(subset = ['comment_count'])

In [ ]:
# fixing the data types
data_final['published_at'] = data_final['published_at'].astype('datetime64[ns]')
data_final['view_count'] = data_final['view_count'].astype('int')
data_final['favorite_count'] = data_final['favorite_count'].astype('int')
data_final['like_count'] = data_final['like_count'].astype('int')
data_final['comment_count'] = data_final['comment_count'].astype('int')

In [ ]:
# checking the schema after applying the above 2 steps
data_final.info()

# 6. Importing the Data into a MySQL database

In [ ]:
# assigning database login info to variables
host = os.getenv('host')
user = os.getenv('user')
password = os.getenv('password')
database = os.getenv('database')

In [ ]:
# using the above variables to build the connection parameter
conn_properties = {
    'host' : host,
    'user' : user,
    'password' : password,
    'database' : database,
    'local_infile' : True,
    'charset' : 'utf8mb4'
}

In [ ]:
# connecting to the database
mysql_conn = sql.connect(**conn_properties)
mysql_cursor = mysql_conn.cursor()

In [ ]:
# create an SQLAlchemy engine to connect to the MySQL database
engine = create_engine("mysql+pymysql://{user}:{password}@{host}/{database}".format(host = host, database = database, user = user, password = password))

In [ ]:
# creating a new table
q1_createtable = 'create table if not exists search (kind varchar(60), video_id varchar(60), published_at datetime, channel_id varchar(60), title varchar(240), description varchar(240), channel_title varchar(60), next_page_token varchar(60), region_code varchar(3), view_count int, like_count int, favorite_count int, comment_count int, primary key(video_id))'
q1_exe = mysql_cursor.execute(q1_createtable)

In [ ]:
# loading the data to the newly-created table   
data_final.to_sql('search', engine, index = False, if_exists = 'replace')